In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
import string
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import ipynb.fs.full.MyDataset as MyDataset
import ipynb.fs.full.Models as Models
import ipynb.fs.full.Utils as Utils
import ipynb.fs.full.RMSELoss as Loss

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        #transforms.Resize(size=400),
        #transforms.RandomRotation(degrees=2),
        #transforms.ColorJitter(),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

transform = image_transforms["train"]


    

'\ntransform = transforms.Compose(\n    [transforms.RandomResizedCrop(224),\n        transforms.RandomHorizontalFlip(),\n        transforms.ToTensor()])\n'

In [4]:
pre_resized=True
if pre_resized:
    csv_file = "data/Ch2_002_resized/interpolated.csv"
    directory = "data/Ch2_002_resized"
else:
    csv_file = "data/Ch2_002/interpolated.csv"
    directory = "data/Ch2_002"  

dataset = MyDataset.MyDataset(csv_file, directory, transform = trans, enable_cache=False, reduce_near_zero_samples=False)


a,b,c = int(0.5*len(dataset)),int(0.25*len(dataset)),int(0.25*len(dataset))
m = len(dataset) - (a+b+c)



train_set, val_set, test_set = torch.utils.data.random_split(dataset, [a+m,b,c])
testloader = DataLoader(dataset, batch_size=1, shuffle=False, )

In [6]:
model = Models.ConvNet2()
model.load_state_dict(torch.load("save_model/mytraining.pt"))

use_cuda = torch.cuda.is_available()
print("Gpu:",use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")

 
if use_cuda:
    model.cuda()
    model.to(device)
    
model.eval()

Gpu: True


ConvNet2(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(3, 3))
  (bnm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 8, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (bnm2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
  (bnm3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bnm4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bnm5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mo

In [10]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch
import torch.nn as nn
import pickle
import os
import argparse
import matplotlib.pyplot as plt
from matplotlib import cm
import PIL
import matplotlib



class CamExtractor():
    """
        Extracts cam features from the model
    """

    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

    def save_gradient(self, grad):
        self.gradients = grad.cpu()
        

    def forward_pass_on_convolutions(self, x):
        """
            Does a forward pass on convolutions, hooks the function at given layer
        """
        x =  self.model.conv1(x)
        x =  self.model.bnm1(x)
        x =  self.model.relu1(x)
        x =  self.model.pool3(x)

        x =  self.model.conv2(x)
        x =  self.model.bnm2(x)
        x =  self.model.relu2(x)
        x =  self.model.pool2(x)
        
        x =  self.model.conv3(x)
        x =  self.model.bnm3(x)
        x =  self.model.relu3(x)
        x =  self.model.pool2(x)
        
        x =  self.model.conv4(x)
        x =  self.model.bnm4(x)
        x =  self.model.relu4(x)
        x =  self.model.pool2(x)
        
        
        x =  self.model.conv5(x)
        x =  self.model.bnm5(x)
        x =  self.model.relu5(x)
        
        x.register_hook(self.save_gradient)
        conv_output = x.cpu()  # Save the convolution output on that layer

        x = x.view(x.shape[0],fl_1_input_dim)
        
        
        
        x = self.model.fc1(x)
        x = F.relu(x)
        x = self.model.dropout1(x)
        
        x = self.model.fc2(x)
        x = F.relu(x)
        x = self.model.dropout2(x)
        
        #print("fc2", x.shape)
        
        x = self.model.fc3(x)
        x = F.relu(x)
        x = self.model.dropout3(x)
        
        x = self.model.fc4(x)
        x = F.relu(x)
        #x = self.dropout4(x)
        
        x = self.model.fc5(x)
        
        return conv_output, x

    def forward_pass(self, x):
        """
            Does a full forward pass on the model
        """
        # Forward pass on the convolutions
        conv_output, x = self.forward_pass_on_convolutions(x)

        return conv_output, x


class GradCam():
    """
        Produces class activation map
    """

    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        # Define extractor
        self.extractor = CamExtractor(self.model, target_layer)

    def generate_cam(self, input_image, angle=None):
        # Full forward pass
        # conv_output is the output of convolutions at specified layer
        # model_output is the final output of the model (1, 1000)
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if angle is None:
            print("No angle given")

        # Zero grads
        self.model.fc5.zero_grad()
        # self.model.classifier.zero_grad()
        # Backward pass with specified target
        model_output.backward(gradient=angle.unsqueeze(1), retain_graph=True)
        
        
        # Get hooked gradients
        guided_gradients = self.extractor.gradients.data.numpy()[0]
        # Get convolution outputs
        target = conv_output.data.numpy()[0]
        # Get weights from gradients
        # Take averages for each gradient
        weights = np.mean(guided_gradients, axis=(1, 2))
        # Create empty numpy array for cam
        cam = np.ones(target.shape[1:], dtype=np.float32)
        # Multiply each weight with its conv output and then, sum
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        cam = cv2.resize(cam, (640, 480))
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) -
                                     np.min(cam))  # Normalize between 0-1
        cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize
        return cam

def save_class_activation_on_image(org_img, activation_map, file_name):
    """
        Saves cam activation map and activation map on the original image
    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): activation map (grayscale) 0-255
        file_name (str): File name of the exported image
    """
    if not os.path.exists('./heatmap'):
        os.makedirs('./heatmap')
    # Grayscale activation map
    #path_to_file = os.path.join('./results', file_name + '_Cam_Grayscale.jpg')
    #cv2.imwrite(path_to_file, activation_map)
    # Heatmap of activation map
    activation_heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_HSV)
    #path_to_file = os.path.join('./results', file_name + '_Cam_Heatmap.jpg')
    #cv2.imwrite(path_to_file, activation_heatmap)
    
    # Heatmap on picture
    #org_img = cv2.resize(org_img, (224, 224))
    img_with_heatmap = np.float32(activation_heatmap) + np.float32(org_img)
    img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)
    path_to_file = os.path.join('./heatmap', file_name )
    cv2.imwrite(path_to_file, np.uint8(255 * img_with_heatmap))
    #cv2.imshow(file_name,np.uint8(255 * img_with_heatmap))
    
    


    
    


In [11]:
i=0
print ("starting heatmap generation")
for data in testloader:


    path = data['Path'][0]

    if "center" in path:
        i+=1
        
        if i > 9998:
            
            inputs = data['X']
            angles = data['y']

            path_ = path.replace("_resized", "")
            name = path.split("/")[-1]

            inputs = inputs.to(device)
            angles = angles.to(device)
            outputs = torch.squeeze(model(inputs))
            # Open CV preporcessing
            image = cv2.imread(path)
            

            grad_cam = GradCam(model, target_layer='')
            # Generate cam mask
            cam = grad_cam.generate_cam(inputs, angles)

            #print(cam)
            # Save mask
            image_ = cv2.imread(path_)
            save_class_activation_on_image(image_, cam, name)
            
        if i%999==0:
            print(i,'Grad cam completed')

        if i%15099==0:
            break
        
    

starting heatmap generation
999 Grad cam completed
1998 Grad cam completed
2997 Grad cam completed
3996 Grad cam completed
4995 Grad cam completed
5994 Grad cam completed
6993 Grad cam completed
7992 Grad cam completed
8991 Grad cam completed
9990 Grad cam completed


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:141: RuntimeWarning: invalid value encountered in true_divide


10989 Grad cam completed
11988 Grad cam completed
12987 Grad cam completed
13986 Grad cam completed
14985 Grad cam completed
